In [2]:
# Linear V4: SVM, TERd
# Linear V3 : Normal, SVM

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve
from sklearn.svm import SVC
from keras.utils import to_categorical
import karnet_v1
from karnet_v1 import KARnet

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\herok\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\herok\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\herok\Anaconda3\lib\site-packages\tensorflow\python\framework\

In [5]:
from siam_utils import *
import siamkar_v2_hp
import karnet_v1

Instructions for updating:
Colocations handled automatically by placer.


In [6]:

PATH = "D:\\Data\\WIFI\\180_100\\"
# save path
res_path = 'D:\\Git\\MILab\\190522_Wisig_KAR\\res\\'


col_df = (['timenow','host','model_type','rseed',
'splits','n_val','fc_size','batch_size','n_iter','alpha','lr','eer','trn_time','params','desc'])


res_df = pd.DataFrame()
roc_df = pd.DataFrame()

desc = 'mining_linear'

types = [0]
seeds = [10,20,30,40,50]

In [7]:
rseed=seeds[0]
splits = 0
np.random.seed(rseed)        
n_splits = 2
data_skf = ReadData(PATH,n_splits,rseed=rseed)

loading data from D:\Data\WIFI\180_100\


In [8]:
data_list = data_skf[splits]

In [67]:
                # Train KARnet
                kar_x = np.squeeze(data_list[0]).reshape([-1,500*30*6])
                kar_labels = [get_key(i,data_list[1]) for i in range(len(kar_x))]
                kar_y = to_categorical(kar_labels)

                kar_xval = np.squeeze(data_list[2]).reshape([-1,500*30*6])
                kar_labels_val = [get_key(i,data_list[3]) for i in range(len(kar_xval))]
                kar_yval = to_categorical(kar_labels_val)

In [68]:
X_d,c_d,Xval_d,cval_d,Y_d,Yval_d = data_list
X_dm = np.mean(X_d,axis=3).reshape([-1,500*6])
Xval_dm = np.mean(Xval_d,axis=3).reshape([-1,500*6])

In [69]:
from hpelm import ELM

In [70]:
X = X_dm
T = kar_y

In [88]:
elm = ELM(X.shape[1], T.shape[1])
elm.add_neurons(1024, "tanh")
elm.add_neurons(16, "tanh")

In [89]:
elm.train(X,T)

Covariance matrix is not full rank; solving with SVD (slow)
This happened because you have duplicated or too many neurons


C:\Users\herok\Anaconda3\lib\site-packages\hpelm\nnets\slfn_python.py:65: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  B = np.linalg.lstsq(HH, HT)[0]


In [90]:
Y = elm.predict(X)
Y.shape

(2000, 100)

In [84]:
aa = np.argmax(Y,axis=1)

In [86]:
aa

array([29, 29, 29, ..., 84, 84, 84], dtype=int64)

In [87]:
Y_d

array([[29],
       [29],
       [29],
       ...,
       [84],
       [84],
       [84]])

In [66]:
elm.confusion(T, Y)

AssertionError: Confusion matrix works only for regression